# RGB-IR Image Process

On this page you can load and interact with RGBI (red, green, blue, and infrared) imagery processed by LJ for Felix's lab.

This is really easy to use. You can clik on a box and hit CTRL-ENTER, or, select a box and hit the Run button above. Or, you can do Cell->RunAll.

What is cool is you can interact with it "in place" (e.g., edit) to get different results

### Packages needed
conda install -c anaconda pillow matplotlib ipywidgets pandas  
conda install -c conda-forge mplcursors opencv scikit-image gdal notebook jupyter_contrib_nbextensions

##### For cell collapsing
1. conda install -c conda-forge jupyter_contrib_nbextensions
2. jupyter contrib nbextension install --user
3. jupyter nbextension enable codefolding/main

In [1]:
# Editor appearance set up

# Change backend for interactive plot
# %matplotlib nbagg
# %matplotlib osx
%matplotlib tk
# %matplotlib qt

# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Extend width of Jupyter Notebook Cell to the size of browser
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

Interactive plot activated


In [3]:
# Every parameter to be modified is here

# Image directory
directory = 'data/'
ext = '.tif'
file_name = 'RGB-IR-G4'
file_name = 'stage2_sfm'
image_file = directory + file_name + ext

# Colormap
cm = 'gist_gray'

# Plot names
pns = ['p1',
       'p2',
       'p3',]
#        'p4',
#        'p5',
#        'p6',
#        'p7',
#        'p8',
#        'p9',
#        'p10']

saved_params = file_name+'_saved_params'

In [2]:
# Import packages needed

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets  import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pickle

# import config as cfg

In [4]:
# Load image and print size & pre-process

# Use skimage to load multi-layer tiff file
rgb_ir_img = io.imread(image_file)
print("Image Shape: ", rgb_ir_img.shape)


# Extract layers from the multilayer tiff file and do some adjustments
layer_RGB = np.copy(rgb_ir_img[:, :, 0:3])
layer_IR = np.copy(rgb_ir_img[:, :, 3])
layer_mask = np.copy(rgb_ir_img[:, :, 4])

# Change from 16 bit to 8 bit(max 65535 to max 255)
layer_RGB = np.round(layer_RGB / 257)
layer_RGB = layer_RGB.astype(int)

# Set background to black (You can choose background color when exporting orthomosaic from agisoft, 
# the default is white)
layer_RGB[np.where(layer_mask == 0)] = 0


# IR values only ranges between arround 7000-8000, so we need to adjust the background to 
# a bit lower than the smallest value of thermal to make it well-spread on colormap
layer_IR[np.where(layer_mask == 0)] = layer_IR.min() #// 100 * 100

Image Shape:  (7690, 12484, 5)


In [ ]:
# RGB to HSV (H 0-1, S 0-1, V 0-255)

layer_HSV = matplotlib.colors.rgb_to_hsv(layer_RGB)

layer_hue = layer_HSV[:, :, 0]
layer_saturation = layer_HSV[:, :, 1]
layer_value = layer_HSV[:, :, 2]

In [16]:
# Interested area pre-selection - Crop out the image for faster process

fig, ax = plt.subplots(figsize=(7, 7))
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
myax = ax.imshow(layer_RGB)

print('Click and drag to select area, press [x] to deactivate seletor, press [c] to continue selection')

def line_select_callback(eclick, erelease):
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    
def toggle_rect_selector(event):
    if event.key in ['X', 'x'] and toggle_rect_selector.RS.active:
        toggle_rect_selector.RS.set_active(False)
        print('Selector deactivated')
    if event.key in ['C', 'c'] and not toggle_rect_selector.RS.active:
        toggle_rect_selector.RS.set_active(True)
        print('Selector activated')

toggle_rect_selector.RS = RectangleSelector(ax, line_select_callback,drawtype='box', useblit=True, button=[1], minspanx=50, minspany=50,
                                            rectprops=dict(facecolor='red', edgecolor='black', alpha=0.1, fill=True), 
                                            spancoords='pixels', interactive=True)

plt.connect('key_press_event', toggle_rect_selector)
plt.show()

Click and drag to select area, press [x] to deactivate seletor, press [c] to continue selection


In [6]:
# Print up-left & bottom-right corners of area of interest

selected_area_corners = toggle_rect_selector.RS.corners
ul_x, ul_y = np.round([selected_area_corners[0][0], selected_area_corners[1][1]])
br_x, br_y = np.round([selected_area_corners[0][2], selected_area_corners[1][2]])

ul_x = ul_x.astype(int)
ul_y = ul_y.astype(int)
br_x = br_x.astype(int)
br_y = br_y.astype(int)

print('Up-left: ', ul_x, ',', ul_y)
print('Bottom-right: ', br_x, ',', br_y)

Up-left:  1922 , 2358
Bottom-right:  4220 , 3825


In [ ]:
# Show IR image & Temperature image

layer_temp = np.asarray(layer_IR) * 0.04 - 273.15 # this was FLIR's recommended transform for DuoProR

fig, axs = plt.subplots(1, 2, figsize=(6, 6))
fig.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95, wspace=0.3)
axs[0].set_title('Temperature (℃)')
axs[1].set_title('RGB')
axim0 = axs[0].imshow(layer_temp[ul_y:br_y, ul_x:br_x], cmap=plt.get_cmap(cm))
axim1 = axs[1].imshow(layer_temp[ul_y:br_y, ul_x:br_x], cmap=plt.get_cmap(cm))
fig.colorbar(axim0, ax=axs[0])
mplcursors.cursor(axs[0])
mplcursors.cursor(axs[1])
axs[1].imshow(layer_RGB[ul_y:br_y, ul_x:br_x, :]) # Show RGB on top of IR

print("Min temp: ", layer_temp.min(), "℃")
print("Max temp: ", layer_temp.max(), "℃")

In [ ]:
# Calculate Vegetation Index and display

# Original formula
# DGCI = ((layer_HSV[:, :, 0] - 60) / 60 + (1 - layer_HSV[:, :, 1]) + (1 - layer_HSV[:, :, 2]/255)) / 3

layer_hue = layer_hue * 360
layer_hue = layer_hue - 60
layer_hue[np.where(layer_hue<0)] = 0
layer_hue[np.where(layer_hue>60)] = 60
print("Adjusted hue min: ", layer_hue.min())
print("Adjusted hue max: ", layer_hue.max())

DGCI = (layer_hue / 60 + (1 - layer_saturation) + (1 - layer_value / 255)) / 3

# Adjust value to show
DGCI[np.where(layer_mask == 0)] = DGCI.min()

fig, ax = plt.subplots(figsize=(5, 5))
fig.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95, wspace=0.3)
plt.title('DGCI')
axim = ax.imshow(DGCI[ul_y:br_y, ul_x:br_x], cmap=plt.get_cmap(cm))
fig.colorbar(axim)

In [14]:
# Set boundary for each plot

print('Here\'s a list of plots, please confirm (y/n)\n', pns)
# user_input = input()

# if user_input != 'y':
#     exit()

plot_flag = 0
plot_name = pns[plot_flag]
plot_vertices = []
max_plot_flag = len(pns)
one_plot_vertices = None
print('\nSelect vertices for', plot_name, '(press [x] to save, press [c] to continue to next plot)')

def onselect(vert):
    global one_plot_vertices
    one_plot_vertices = np.round(np.array(vert))

def toggle_selector(event):
    global plot_flag, plot_name, max_plot_flag, one_plot_vertices, cid
#     global max_plot_flag
    if event.key in ['X', 'x'] and toggle_selector.PS.active:
        toggle_selector.PS.set_active(False)
        ################max_plot_flag save point ##################
        plot_vertices.append(one_plot_vertices)
        print('Polygon vertices saved')
        print(one_plot_vertices, '\n')
        plot_flag += 1
        if plot_flag == max_plot_flag:
            print('All plots\'re selected')
            plt.disconnect(cid)
    if event.key in ['C', 'c'] and not toggle_selector.PS.active:
        print('Next plot')
        toggle_selector.PS = PolygonSelector(ax, onselect, useblit=True)
        plot_name = pns[plot_flag]
        print('\nSelect vertices for', plot_name, '(press [x] to save, press [c] to continue to next plot)')


fig, ax = plt.subplots(figsize=(10, 10))
fig.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
ax.imshow(layer_RGB[ul_y:br_y, ul_x:br_x, :])


toggle_selector.PS = PolygonSelector(ax, onselect, useblit=True)

cid = plt.connect('key_press_event', toggle_selector)
plt.show()


Here's a list of plots, please confirm (y/n)
 ['p1', 'p2', 'p3']

Select vertices for p1 (press [x] to save, press [c] to continue to next plot)
Polygon vertices saved
[[385. 273.]
 [865. 307.]
 [970. 539.]
 [768. 715.]
 [314. 692.]] 

Next plot

Select vertices for p2 (press [x] to save, press [c] to continue to next plot)
Polygon vertices saved
[[1057.  322.]
 [1378.  756.]
 [1018.  508.]] 

Next plot

Select vertices for p3 (press [x] to save, press [c] to continue to next plot)
Polygon vertices saved
[[1521.  567.]
 [1623.  409.]
 [1840.  396.]
 [2009.  434.]
 [2093.  575.]
 [2101.  766.]
 [2096.  902.]
 [1989.  981.]
 [1899. 1103.]
 [1792. 1032.]
 [1840.  904.]
 [1590.  960.]
 [1557.  838.]
 [1437.  782.]] 

All plots're selected


In [17]:
# Calculate & save GPS coordinates of each plot

ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()

plot_vertices_gps = []
for i in range(max_plot_flag):
    one_plot_vertices = plot_vertices[i]
    one_plot_vertices_gps = []
    for vertex in one_plot_vertices:
        x = ul_x + vertex[0]
        y = ul_y + vertex[1]
        
        xgeo = gt[0] + x*gt[1] + y*gt[2]
        ygeo = gt[3] + x*gt[4] + y*gt[5]
        one_plot_vertices_gps.append([xgeo, ygeo])
    one_plot_vertices_gps = np.array(one_plot_vertices_gps)
    print('Plot', i, '\n', one_plot_vertices_gps, '\n')
    plot_vertices_gps.append(one_plot_vertices_gps)
    

with open(saved_params, 'wb') as f:
    pickle.dump(plot_vertices_gps, f)
print('GPS coordinates saved to', saved_params)

Plot 0 
 [[-92.35534137  38.92792965]
 [-92.35526389  38.92792536]
 [-92.35524694  38.92789611]
 [-92.35527955  38.92787392]
 [-92.35535283  38.92787682]] 

Plot 1 
 [[-92.3552329   38.92792347]
 [-92.35518108  38.92786875]
 [-92.35523919  38.92790002]] 

Plot 2 
 [[-92.355158    38.92789258]
 [-92.35514153  38.9279125 ]
 [-92.35510651  38.92791414]
 [-92.35507923  38.92790935]
 [-92.35506567  38.92789157]
 [-92.35506438  38.92786748]
 [-92.35506518  38.92785034]
 [-92.35508245  38.92784038]
 [-92.35509698  38.92782499]
 [-92.35511425  38.92783395]
 [-92.35510651  38.92785008]
 [-92.35514686  38.92784302]
 [-92.35515219  38.92785841]
 [-92.35517156  38.92786547]] 

GPS coordinates saved to stage2_sfm_saved_params
